In [1]:
from biosimulator_processes.processes.copasi_process import CopasiProcess
from biosimulator_processes import CORE 
from biosimulator_processes.processes.instance import generate_ode_process_instance
from process_bigraph import Step
from process_bigraph.experiments.parameter_scan import RunProcess

CobraProcess registered successfully as cobra.

CopasiProcess registered successfully as copasi.

SmoldynProcess registered successfully as smoldyn.

TelluriumProcess registered successfully as tellurium.

AmiciProcess registered successfully as amici.

Available processes:
[ 'console-emitter',
  'ram-emitter',
  'cobra',
  'copasi',
  'smoldyn',
  'tellurium',
  'amici']
GetSbmlStep not available. Error: No module named 'biosimulator_processes.steps.get_sbml'


In [2]:
fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Varusai-Sci-Rep-2018-mTOR-signaling-LSODA-LSODAR-SBML/Varusai2018.xml'
obs1 = ['floating_species_concentrations', 'Akt']
proc_addresses = ['copasi', 'tellurium', 'amici']
proc_address = 'amici'
step = 0.01
dur = 100

In [3]:
amici_process = generate_ode_process_instance(entrypoint=fp, process_address=proc_address, step_size=step, duration=dur, observables=obs1)

2024-05-24 17:56:34.901 - amici.sbml_import - INFO - Finished importing SBML                         (3.12E-02s)
2024-05-24 17:56:34.954 - amici.sbml_import - INFO - Finished processing SBML observables            (5.09E-02s)
2024-05-24 17:56:34.956 - amici.sbml_import - INFO - Finished processing SBML event observables      (4.17E-07s)
2024-05-24 17:56:34.993 - amici.de_model - INFO - Finished computing xdot                            (2.98E-03s)
2024-05-24 17:56:34.996 - amici.de_model - INFO - Finished computing x0                              (9.81E-04s)
2024-05-24 17:56:35.030 - amici.de_model - INFO - Finished computing w                               (3.17E-02s)
2024-05-24 17:56:35.543 - amici.de_export - INFO - Finished generating cpp code                      (5.09E-01s)
2024-05-24 17:56:44.895 - amici.de_export - INFO - Finished compiling cpp code                       (9.35E+00s)


In [9]:
amici_process.update({})

KeyboardInterrupt: 

In [5]:
def run_ode_comparison(process_addresses, model_fp, step, dur, observables):
    processes = [generate_ode_process_instance(entrypoint=model_fp, process_address=address, step_size=step, duration=dur, observables=observables) for address in process_addresses]
    print(processes)
    
    return [process.run({}) for process in processes]
    

run_ode_comparison(proc_addresses, fp, step, dur, obs1)

2024-05-24 17:44:21.913 - amici.sbml_import - INFO - Finished loading SBML                           (1.07E-02s)
2024-05-24 17:44:21.955 - amici.sbml_import - INFO - Finished importing SBML                         (3.94E-02s)
2024-05-24 17:44:21.974 - amici.sbml_import - INFO - Finished processing SBML observables            (1.59E-02s)
2024-05-24 17:44:21.978 - amici.sbml_import - INFO - Finished processing SBML event observables      (3.75E-07s)
2024-05-24 17:44:22.010 - amici.de_model - INFO - Finished computing xdot                            (3.25E-03s)
2024-05-24 17:44:22.015 - amici.de_model - INFO - Finished computing x0                              (2.35E-03s)


found a filepath


2024-05-24 17:44:22.029 - amici.de_model - INFO - Finished computing w                               (1.06E-02s)
2024-05-24 17:44:22.632 - amici.de_export - INFO - Finished generating cpp code                      (5.99E-01s)
2024-05-24 17:44:30.652 - amici.de_export - INFO - Finished compiling cpp code                       (8.02E+00s)
/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/biosimulator_processes/processes/copasi_process.py:194: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



[<biosimulator_processes.processes.instance.ODEProcess object at 0x36b77b3a0>, <biosimulator_processes.processes.instance.ODEProcess object at 0x36b779840>, <biosimulator_processes.processes.instance.ODEProcess object at 0x36d1281c0>]


KeyboardInterrupt: 